In [99]:
import os
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import audtorch
import torch
from torch import nn
from torch.nn.utils.rnn import *
import matplotlib.pyplot as plt

In [100]:
!nvidia-smi

Wed Dec  2 22:59:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    38W / 300W |   1451MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Connecting GDrive

In [101]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [102]:
#%cd /content/gdrive/My\ Drive/Colab/IDL_Final_Project  
%cd /content/gdrive/My\ Drive/IDL_Final_Project   

/content/gdrive/.shortcut-targets-by-id/1cLTX5b-YNPmc2AuVB16CsNFPAABZZ53P/IDL_Final_Project


## Loading data

In [103]:
!ls

 avg_age_loss_lr1_0.0001_data_4610.png	 loss.png
 avg_age_loss_lr1_0.001_data_4610.png	 Model_avg_age_1606450302.1992056.model
 avg_age_loss_lr1_0.1_data_4610.png	 Model_avg_age_1606468211.1750672.model
'Copy of fisher'			 Model_avg_age_1606469253.3927295.model
 dataProcessing.ipynb			 Model_avg_age_1606469458.7631238.model
 doc					 Model_avg_age_1606469563.55613.model
 FinalProject-CNNs.ipynb		 Model_avg_age_1606486161.641004.model
 FinalProjectDataPrep.ipynb		 Model_avg_age_1606554191.5310152.model
 FinalProject-dNNs.ipynb		 new
'Final Project Draft.gdoc'		 pins
'Final Project.gdoc'			 pins.zip
 FinalProject.ipynb			 sox
 fisher					 test
 fisher_wav				 test_data.npy
 loss_lr_0.001_data_4610.png		 test_labels.npy
 loss_lr_0.001_data_461.png		 timit
 loss_lr_0.01_data_4610.png		 train
 loss_lr1_0.001_data_4610.png		 train_data.npy
 loss_lr1_0.001_data_461.png		 train_labels.npy
 loss_lr_1e-05_data_4610.png		 vad_wavs


In [104]:
train_data = np.load("train_data.npy",allow_pickle=True)  
train_labels = np.load("train_labels.npy",allow_pickle=True)

test_data = np.load("test_data.npy",allow_pickle=True)  
test_labels = np.load("test_labels.npy",allow_pickle=True)

# Labels columns: age, height, gender
# age: years, height: centimeters, gender: 1-male, 0-female
# train_labels[0]
# >> array([27.  ,  167.64,    0.  ])

Normalizing training Labels to have zero mean and unit standard deviation

In [105]:
train_labels[0]

array([ 27.  , 167.64,   0.  ])

In [106]:
tr_labels = train_labels.copy()

avg_age = tr_labels[:,0].mean()
std_age = tr_labels[:,0].std()

tr_labels[:,0] = (tr_labels[:,0] - avg_age)/std_age

# tes_labels[:,:-1] = (tes_labels[:,:-1] - tes_labels[:,:-1].mean(axis=0))/tes_labels[:,:-1].std(axis=0)

In [107]:
# Shuffling data
train_indices = np.random.permutation(len(train_data))
train_data = train_data[train_indices]
train_labels = train_labels[train_indices]

test_indices = np.random.permutation(len(test_data))
test_data = test_data[test_indices]
test_labels = test_labels[test_indices]

In [108]:
male_train_labels = tr_labels[tr_labels[:,-1] == 1]
m_tr_avg_age = male_train_labels[:,0].mean()

female_train_labels = tr_labels[tr_labels[:,-1] == 0]
f_tr_avg_age = female_train_labels[:,0].mean()

print("Female's avg age: ", f_tr_avg_age)
print("Male's avg age: ", m_tr_avg_age)

Female's avg age:  -0.056530944733449814
Male's avg age:  0.023410053800661697


In [109]:
male_test_labels = test_labels[test_labels[:,-1] == 1]
m_test_avg_age = male_test_labels[:,0].mean()

female_test_labels = test_labels[test_labels[:,-1] == 0]
f_test_avg_age = female_test_labels[:,0].mean()

print("Female's avg age: ", m_test_avg_age)
print("Male's avg age: ", f_test_avg_age)

Female's avg age:  31.229609929078013
Male's avg age:  31.045694200351495


In [110]:
# Changing age to the average
# tr_labels = train_labels.copy()

# for idx in range(tr_labels.shape[0]):
  
#   if tr_labels[idx, -1] == 1:
#     tr_labels[idx, 0] = m_avg
#   else:
#     tr_labels[idx, 0] = f_avg

tes_labels = test_labels.copy()
for idx in range(tes_labels.shape[0]):
  
  if tes_labels[idx, -1] == 1:
    tes_labels[idx, 0] = m_tr_avg_age
  else:
    tes_labels[idx, 0] = f_tr_avg_age

## Dataset

In [111]:
class CustomDataset(Dataset):
  def __init__(self,data,labels=None):

    # lens = [utter.shape[0] for utter in data]
    max_lens = 336 #max(lens) # The longest utterance in training data has 336 frames

    pad_above = 0; pad_below = 0
    pad_left = 0; pad_right = 0
    
    self.X = []

    for utter in data:
      utter = np.pad(utter, pad_width=((pad_above, max_lens-utter.shape[0]),(pad_left, pad_right)))
      utter = torch.from_numpy(utter)
      utter = utter.T  # (64, 336)

      self.X.append(utter.type(torch.float32))

    # self.X = [torch.from_numpy(d) for d in data]

    if labels is not None:
      self.y = torch.from_numpy(labels) 
  
  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    X = self.X[idx]

    if self.y is not None:
      y = self.y[idx]

      return X, y
    else:
      return X, None

## Dataloader

### DataLoaders

In [112]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training
batch_size = 64

train_dataset = CustomDataset(train_data, tr_labels) #train_labels
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, 
                          num_workers=4,drop_last=True) 

# Validation
val_dataset = CustomDataset(test_data, tes_labels)  #test_labels: age is the average age for corresponding to the gender in training data
validation_loader = DataLoader(val_dataset, shuffle=False,batch_size=batch_size, 
                               num_workers=4, drop_last=True)

In [113]:
train_dataset.__getitem__(0)[0].size()

torch.Size([64, 336])

In [114]:
 
# conv 32, relu, conv 64, relu, conv 128 relu,  meanpool (down to a 128-d vector),  linear layre for prediction.
# cnn1 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1, bias=False)  # #([64, 32, 325])

# cnn2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, stride=2, bias=False) # #([64, 64, 163])

# cnn3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=2, stride=2, bias=False) #([64, 128, 81])

# pool = nn.AvgPool1d(2, stride=2)  #([64, 128, 40])

# CONV 2D
cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=2, stride=2, bias=False)  # ([64, 32, 63, 335])

cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=1, stride=1, bias=False) # ([64, 64, 32, 168])

cnn3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=1, stride=1, bias=False) # ([64, 128, 16, 84])

pool = nn.AvgPool2d((3,3))  # ([64, 128, 8, 42])

# linear = nn.Linear(128 * 21 * 111, 1)
# nn.BatchNorm2d(channel_out)

for inputs, labels in train_loader:
  inp = inputs.unsqueeze(1)  # ([64, 64, 327])

  print("in",inp.size())

  out = cnn1(inp)  #([64, 32, 325])
  print("CNN1 out", out.size())

  out = cnn2(out)  #([64, 64, 163])
  print("CNN2 out", out.size())

  out = cnn3(out)  #([64, 128, 81])
  print("CNN3 out", out.size())

  out = pool(out)  #([64, 128, 40])
  print("Pool out", out.size())

  # out = out.view(out.size(0), -1)
  # out = linear(out)
  # print("Linear out", out.size())

  break

in torch.Size([64, 1, 64, 336])
CNN1 out torch.Size([64, 32, 32, 168])
CNN2 out torch.Size([64, 64, 32, 168])
CNN3 out torch.Size([64, 128, 32, 168])
Pool out torch.Size([64, 128, 10, 56])


## Model

In [115]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # conv 32, relu, conv 64, relu, conv 128 relu,  meanpool (down to a 128-d vector),  linear layre for prediction.
        dropout = torch.nn.Dropout(p=0.1)
        hiddenLayer1Size=512
        hiddenLayer2Size=int(hiddenLayer1Size/4)
        hiddenLayer3Size=int(hiddenLayer1Size/8)
        hiddenLayer4Size=int(hiddenLayer1Size/16)
        hiddenLayer5Size=int(hiddenLayer1Size/32)

        #Neural Network layers
        linear1=torch.nn.Linear(64, hiddenLayer1Size, bias=True) 
        linear2=torch.nn.Linear(hiddenLayer1Size, hiddenLayer2Size)
        linear3=torch.nn.Linear(hiddenLayer2Size, hiddenLayer3Size)
        linear4=torch.nn.Linear(hiddenLayer3Size, hiddenLayer4Size)
        linear5=torch.nn.Linear(hiddenLayer4Size, hiddenLayer5Size)
        linear6=torch.nn.Linear(hiddenLayer5Size, 1)
        sigmoid = torch.nn.Sigmoid()
        threshold = nn.Threshold(0.5, 0)
        tanh=torch.nn.Tanh()
        relu=torch.nn.LeakyReLU()

        self.net = torch.nn.Sequential(linear1,nn.BatchNorm1d(hiddenLayer1Size),relu,
                          linear2,dropout,relu,
                          linear3,dropout,relu,
                          linear4,dropout,relu,
                          linear5,dropout,relu,
                          linear6,dropout,relu,
                          sigmoid)
    
    def forward(self, batch_data): 
      #print("batch",batch_data.shape)

      batch_data = batch_data.reshape(-1,batch_data.shape[0])
      #print("batch",batch_data.shape)
      out = self.net(batch_data) #1D CNN
      # out = self.cnns(batch_data.unsqueeze(1))  # 2D CNN
      
      #out = self.fc(out.view(out.size(0), -1))
      #print('out',out.)
      return out.view(-1,64)


def init_weights(m):
  if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
    nn.init.kaiming_normal_(m.weight.data, mode='fan_out', nonlinearity='relu')
     
  elif isinstance(m, (nn.BatchNorm1d, nn.GroupNorm)):
    nn.init.constant_(m.weight, 1)
    nn.init.constant_(m.bias, 0)

## Training

### Validate

In [116]:
def validate(model,validation_loader):
  model.eval()
  running_loss_v = 0.0
  correct_v = 0
  total_v = 0

  with torch.no_grad():
    
    for batch_idx, (X, y) in enumerate(validation_loader):
      
      X = X.to(device)
      y = y.to(device)

      age_prediction= model(X)
      

      loss = criterion(age_prediction,y[:,0])

      preds = age_prediction.cpu().numpy()

      # loss2 = criterion(height_prediction,y[:,1])

      # loss = loss1 + loss2  # Adding for MTL

      running_loss_v += loss.item()

      # print('Batch val loss', loss.item())
      total_v += y.size(0)


      torch.cuda.empty_cache()
      del X; del y
      del age_prediction #; del height_prediction
      # del loss1; del loss2
      del loss


  avg_loss_v = running_loss_v / len(validation_loader)

  return avg_loss_v

### Train method

In [117]:
def train():

  total_loss = []
  running_loss = 0.0
  total = 0

  val_avg_loss = []
  train_avg_loss_list = []

  for epoch in range(NUM_EPOCHS):
    print("EPOCH #",epoch + 1)
    
    model.train()  # Re-enabling training
    
    start = time.time()

    for batch_idx, (X, y) in enumerate(train_loader):

      optimizer.zero_grad()

      X = X.to(device)
      y = y.to(device)

      t0 = time.time()

      age_prediction= model(X)
      #print("out",age_prediction.shape,y[:,0].shape)
      
      loss = criterion(age_prediction,y[:,0])

      running_loss += loss.item()
      total += y.size(0)


      if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
        print('[Batch {}]\tLoss: {}'.format(batch_idx + 1, loss.item()))

      t1 = time.time()

      loss.backward()

      t2 = time.time()
      
      if batch_idx == 0 and epoch == 0:
        print("Model Prediction time: ", t1 - t0)
        print("Loss Backward computation time: ", (time.time() - t2))

      optimizer.step()
      # scheduler.step()
      scheduler.step(loss)


      torch.cuda.empty_cache()
      del X; del y
      del age_prediction  #; del height_prediction
      # del loss1; del loss2
      del loss

    train_avg_loss = running_loss / total

    # Validating
    t0 = time.time()
    validation_avg_loss = validate(model,validation_loader)
    running_loss/= len(train_loader)
    total_loss.append(running_loss)
    train_avg_loss_list.append(train_avg_loss)

    val_avg_loss.append(validation_avg_loss)

    if epoch == 0:
      print("Validation time: ", (time.time() - t0))
      print("Epoch duration time: ", (time.time() - start))
      print("*" * 30)
      print("")

    print('Epoch [%d] [train] loss: %.8f' %(epoch + 1, train_avg_loss))
    print('[running] loss: %.8f' % (running_loss))
    print('[valid] loss: %.8f' % (validation_avg_loss))
    print("-" * 30)

    if len(val_avg_loss) >= 2 and val_avg_loss[-1] > val_avg_loss[-2]:
      print("Stopped. Model started overfitting data")
      break

  return val_avg_loss

### main

In [118]:
NUM_EPOCHS = 20 #100

learning_rate = 1e-01  # Best with ADAM: 1e-01

input_size = 64

model = Model()
model.apply(init_weights)

model.to(device)

criterion = nn.L1Loss() #nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay=5e-06)
# optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
    factor=0.1, patience=20, threshold=0.01, verbose=True)

val_avg_loss = train()

EPOCH # 1


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:94: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([336, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Model Prediction time:  0.006265878677368164
Loss Backward computation time:  0.00018596649169921875
[Batch 10]	Loss: 0.8365220420579991
[Batch 20]	Loss: 0.750825555556752
[Batch 30]	Loss: 0.8154957302980123
[Batch 40]	Loss: 0.8119105350818775
[Batch 50]	Loss: 0.707067179910568
Epoch    50: reducing learning rate of group 0 to 1.0000e-02.
[Batch 60]	Loss: 0.8515476944109405
[Batch 70]	Loss: 0.6584040858404769
Epoch    71: reducing learning rate of group 0 to 1.0000e-03.
[Batch 72]	Loss: 0.7468632052465399
Validation time:  0.45340490341186523
Epoch duration time:  2.580939531326294
******************************

Epoch [1] [train] loss: 0.01199426
[running] loss: 0.76763261
[valid] loss: 0.03449679
------------------------------
EPOCH # 2
[Batch 10]	Loss: 0.8356969317167589
[Batch 20]	Loss: 0.8572585638456595
[Batch 30]	Loss: 0.8143899207681444
Epoch   111: reducing learning rate of group 0 to 1.0000e-04.
[Batch 40]	Loss: 0.8509373181285939
[Batch 50]	Loss: 0.5621194349204788
[Batch 60

In [119]:
from time import time
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr': learning_rate,
            'hidden_size': hidden_size,
            'dropout': dropout,
            'layers': nlayers,
            'val_loss':val_avg_loss[-1] 
        }, "Model_avg_age_"+str(time())+".model")

NameError: ignored

In [ ]:
fig = plt.gcf()

plt.plot(list(range(1,len(val_avg_loss) + 1)), val_avg_loss)
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.title('Validation loss')

plt.show()
plt.draw()
# fig.savefig('avg_age_loss_lr1_{}_data_{}.png'.format(learning_rate,len(train_labels)), dpi=100)

## L1 Loss and L2 Loss on Test data

In [ ]:
model.eval()

predictions = []
targets = []

with torch.no_grad():
  for batch_idx, (X, y) in enumerate(validation_loader):

    X = X.to(device)

    output = model(X)

    pred = output.cpu().numpy()

    predictions.extend(pred.tolist())
    targets.extend(y[:,[0,-1]].numpy().tolist())

    torch.cuda.empty_cache()
    del X;
    del output

In [ ]:
predictions = torch.Tensor(predictions)
targets = torch.Tensor(targets)

mae_loss = nn.L1Loss() 
mse_loss = nn.MSELoss()

l1 = mae_loss(predictions, targets[:,0])
l2 = mse_loss(predictions, targets[:,0])

print("MAE: ", l1.item())
print("MSE: ", l2.item())

### MALE

In [ ]:
index = targets[:,1] == 1

l1 = mae_loss(predictions[index], targets[index][:,0])
l2 = mse_loss(predictions[index], targets[index][:,0])

print("Avg: ", predictions[index].numpy().mean())
print("MAE: ", l1.item())
print("MSE: ", l2.item())

### FEMALE

In [ ]:
index = targets[:,1] == 0

l1 = mae_loss(predictions[index], targets[index][:,0])
l2 = mse_loss(predictions[index], targets[index][:,0])

print("Avg: ", predictions[index].numpy().mean())
print("MAE: ", l1.item())
print("MSE: ", l2.item())

## Losses with Actual Test labels

In [ ]:
validation_dataset = CustomDataset(test_data, test_labels)#test_labels
test_loader = DataLoader(validation_dataset, shuffle=False, 
                               batch_size=batch_size, num_workers=4)

In [ ]:
model.eval()

predictions = []
targets = []

with torch.no_grad():
  for batch_idx, (X, y) in enumerate(test_loader):

    X = X.to(device)

    output = model(X)

    pred = output.cpu().numpy()

    predictions.extend(pred.tolist())
    targets.extend(y[:,[0,-1]].numpy().tolist())

    torch.cuda.empty_cache()
    del X;
    del output

In [ ]:
# avg_age = test_labels[:,0].mean()  #test_labels
# std_age = test_labels[:,0].std()

# avg_age = tr_labels[:,0].mean()
# std_age = tr_labels[:,0].std()
avg_age

In [ ]:
pred =(predictions + avg_age) #* std_age

print(min(pred))
print(max(pred))
# predictions

In [ ]:
predictions = torch.Tensor(pred) #predictions
targets = torch.Tensor(targets)

mae_loss = nn.L1Loss() 
mse_loss = nn.MSELoss()

### Male

In [ ]:
index = targets[:,1] == 1

l1 = mae_loss(predictions[index], targets[index][:,0])
l2 = mse_loss(predictions[index], targets[index][:,0])

print("MAE: ", l1.item())
print("MSE: ", l2.item())

### Female

In [ ]:
index = targets[:,1] == 0

l1 = mae_loss(predictions[index], targets[index][:,0])
l2 = mse_loss(predictions[index], targets[index][:,0])

#print("Avg: ", predictions[index].numpy().mean())
print("MAE: ", l1.item())
print("MSE: ", l2.item())